# Computer Vision - Advanced Ideas

## Techniques and Architectures

### Architecture
- **Depthwise Convolutions:** Instead of convolving a kernel across all channels together, we convolve a separate kernel across each channel individually and stack the results together.
- **Depthwise Separable Convolutions:** A more compute-efficient way to compute a convolution, by first using a depthwise convolution, and then using pointwise (1x1) convolutions on the stacked depthwise outputs to create the desired output channels.
- **Squeeze Excitations:** A way of letting the network decide which channels are most important to pass forward to the next layer. Uses a separate $$\text{AvgPool} \rightarrow \text{Linear} \rightarrow \text{ReLU} \rightarrow \text{Linear} \rightarrow \text{Sigmoid}$$ block on each input channel separately to calculate a weight, which is then used to weight that input channel before it passes forward.
- **Inverted Residual (MBConv) Block:** A residual block that uses an inverted bottleneck to blow up the input channels and then shrink then shrink them back down to the output channels. This is opposite of what regular residual blocks do, which is to shrink and then blow back up the channels.

### Data Augmentation
- **Cutmix:** Replace a patch of one image with the patch of another image, creating a sort of hybrid.
    - Must also soften the label for the image too by adding in a proportional amount of the other image label.
- **Mixup:** Take a linear combination of two images with one much brighter than the other.
    - Also want to soften the labels in proportion to the combination.
    - Forces the model to learn to discriminate between image objects better.
- **RandAugment:** Applies a set of common data augmentations at random (rotate, contrast, translate, shear, etc).
- **Random Resized Crop:** Applies a random crop, but also adjusts the aspect ratio by a random amount.
- **Random Erasing:** Blot out a random rectangular patch of the image. Helps the model be more robust to occlusion.

### Training Time
- **Label Smoothing:** Label each output correctly with probability $p=1-\varepsilon$, else use a soft label $\big(\frac{\varepsilon}{K-1}, 1-\varepsilon\big)$.
    - Introduces noise into the labels, which helps the model become more robust and generalize better
- **Stochastic Depth:** Randomly skip layers (or blocks) deeper in the network during training with some probability
    - To implement, just add a Bernoulli RV next to that layer's input activation, which will be 1 most of the time but 0 some of the time
    - This probability should be higher for later layers than earlier layers
    - The skip connection is always allowed to pass through, which is what allows us to drop layers in the middle of the network and not lose much
    - Like dropout and norm, will want to turn this off during inference
- **Progressive Image Resizing:** Progressive Resizing works by initially training on downsampled images, and gradually growing them back to their full size over the course of training. 
    - Reduces costs early in training when the network may focus on learning coarse-grained features that don't require much image resolution.
    - Example: Train for a total of 10 epochs, starting with image size 224x224 for the first 6 epochs, then finetune using image size 256x256 for the last 4 epochs.

### Test Time

- **Test Time Augmentation (TTA):** Use data augmentation at test time to make multiple predictions against augmented version of the same image, and return their average as the "true" prediction. 
    - Helps make the model more robust at test time.

## Object Detection and Segmentation
- **Intersection over Union (IoU):** A way of determining tightness of overlap of multiple bounding boxes. A high IoU means tight overlap, while a low IoU means low overlap. Given by
$$\text{IOU} = \frac{\text{area}(\cap_n \text{boxes})}{\text{area}(\cup_n \text{boxes})}.$$
- **Average Precision:** Area under the precision-recall curve for each class, where the true class and predicted class must be within a given IoU to count
    - A high AP for a given class indicates that predictions and outputs are both accurate and their bounding boxes are almost the same
    - $mAP$: The APs of each class all averaged together, often a "final" reporting metric for object detection
    - $AP_{50}$: The mAP within an output/prediction IoU of 0.5
    - $AP_S$: The mAP of all "small objects" ($area < 32^2$)
    - $AP[.5:.05:.95]$: The mAP at IOUs 0.5...0.95 (in steps of 0.5) all averaged together
- **Dice Loss:** F1 score between the true mask and predicted mask in a segmentation problem
- **Focal Loss:** A modified loss to address class imbalance in object detection. Works by applying a modulating term to each class in the cross entropy, designed to force the model to work harder during training to get better at recognizing the harder training examples. Given by
$$L(\hat y, y) =  - \sum \color{red}{(1 - \hat y_k)^\gamma} y_k \log(\hat y_k),$$
for some hyperparameter $\gamma \geq 0$. A higher $\gamma$ (e.g. $\gamma=5$) means more focus on the harder examples, as for those $(1-\hat y_k) > 0.5$.

## Papers

### ConvNext: A ConvNet for the 2020s (2022)

- Takes Resnets and incorporates modern CV training techniques + transformer design choices
- New training techniques: Modern CV training techniques
    - Optimizer/Scheduler: AdamW and cosine annealing with linear warmup
    - Data augmentation: mixup (0.8), cutmix (1.0),  randaugment (9, 0.5), random erasing (0.25)
    - Regularization: stochastic depth (p=0.1, 0.4, 0.5, 0.5), label smoothing (eps=0.1)
- Macro design: Brought over from Swin transformers
    - Stage compute ratio (# blocks): Resnet # blocks (3, 4, 6, 3) -> Swin # blocks (3, 3, 9, 3)
    - Patchifying (first layer): 7x7 conv stride 2 + max-pool -> 4x4 conv stride 4 (=> non-overlapping convs)
- ResNeXT-ify: Bring over ResNeXT Innovations (more efficiency in terms of flops/acc)
    - Depthwise convolutions (group convolutions): Used as a bottleneck layer in each block (reduces flops)
    - Width up: Network width (# channels in a block) goes from Resnet 64 -> Swin 96 (get acc back up after reduced flops)
- Inverted bottleneck: Changes blocks from [1x1 (384->96), d3x3 (96->96), 1x1 (96->384)] to [d3x3 (96->96), 1x1 (96->384), 1x1 (384->96)]
    - Inversion: Expand middle layer to create inverted bottleneck instead of bottleneck (transformer idea, reduces network flops)
- Larger kernel sizes: Larger kernel sizes mimic self attention by enabling larger receptive fields
    - Move depthwise conv to the first layer of a block instead of the middle (allows for larger kernel sizes w/ fewer channels)
    - Leave the 1x1 convs in the inverted bottleneck to do the heavy computation (i.e. large # channels)
    - Change from the original 3x3 VGG kernel size to larger kernel sizes of 5x5, 7x7, 9x9, or 11x11
    - Performance gains seem to saturate at kernel sizes of 7x7 (which is thus used in the convnext architecture)
- Micro design: change design inside the residual blocks
    - Activation function: ReLU -> GELU (doesn't really make much difference)
    - Fewer activation functions: no more activations after every conv, only between 1x1 conv blocks (transformer idea)
    - Fewer norms: no more norms after every conv, only one before first 1x1 conv (transformer idea)
    - Norm: batch norm -> layer norm (slight improvement, easier to train)
    - Separate downsampling layers: Don't do the downsampling in the blocks, but in separate layers connecting them (which also pass through a LN)
- ConvNext sizes: T tiny (29M), S small (50M), B base (89M), L large (198M), XL x-large (350M)

<img src=https://user-images.githubusercontent.com/8370623/180626875-fe958128-6102-4f01-9ca4-e3a30c3148f9.png width=700>

### EfficientNetV2: Smaller Models and Faster Training (2021)

- Like the original EfficientNet, EfficientNetV2 uses neural architecture search combined with scaling laws to try to find efficient image model architectures.
- The paper claims V2 is significantly more efficient than the original in terms of the accuracy per parameter tradeoff. Its 28M parameter model is comparable in accuracy to the V1 43M parameter model, but 4x faster.
- Replaces MBConv (inverted residual) blocks in the early layers with Fused MBConv blocks. The idea of the Fused MBConv is to replace $\text{Conv}(1\times 1, c=c_{out}) \rightarrow \text{DepthwiseConv}(3\times 3, c=c_{out})$ with just a single convolution $\text{Conv}(3\times 3, c=c_{out})$. 
    - Designed to address the fact that depthwise convolutions in early network layers make training run slower on modern GPUs.
    <img src=https://user-images.githubusercontent.com/37800725/115514568-cfdc4900-a2b6-11eb-8adc-de6dc1f2c993.png width=300>
- Use progressive image resizing to speed up training, as training V1 on large images is slow. As training progresses, image sizes start small and grow larger, while regularization (data augmentation, dropout) starts weak and grows stronger.
- Use a different scaling law for each block in the network, not one scaling law for the whole network.
- Currently close to SOTA on image classification, on par with ConvNeXT. Using in practice is trickier though, as unlike ConvNeXTs, EfficientNets are finnicky to finetune.

<img src=https://production-media.paperswithcode.com/methods/Screen_Shot_2021-09-17_at_9.27.01_AM.png width=400>

### EfficientDet: Scalable and Efficient Object Detection (2020)

- An extension of the EfficentNet architectures to object detection, allowing for the benefits of compound scaling of image resolution/depth/width in detection tasks
- Designed to be a fast/efficient, but accurate set of object detection models on par with the YOLO models
- Uses EfficientNet backbones (B0-B7) to extract features, then feeds those through a BiFPN to extract more features, then feeds those separately through a class prediction network and a bbox prediction network
- A feature pyramid network (FPN) works by taking in sequences of feature maps of decaying sizes. Each feature map interacts with all feature maps of lower size (called a *feature pyramid*) through a sequence of layers. This allows the network to learn to recognize objects at different scales.
- A BiFPN is an FPN that allows associations in both directions, low to high and high to low.
- Comes in sizes D0-D7 (one for each backbone), varying in size from 3.9M-77M parameters.

<img src=https://production-media.paperswithcode.com/methods/Screen_Shot_2020-06-13_at_3.09.28_PM.png width=600>

### YOLOv4 (2020)

- Architecture
    - backbone: CSPDarknet53
    - neck: SPP Block + PANet
    - head: yolov3 (anchor based detection steps, and three levels of detection granularity)
- Bag of freebies: performance improvements with no added inference time
    - mosaic data augmentation (mesh 4 images together and detect objects, helps ignore surroundings)
    - cutmix data augmentation (overlay cut of one image on top of another)
    - label smoothing (dull hard one-hot labels to allow some uncertainty and improve loss comparisons)
    - self-adversarial training (keep model from over-relying on any one part of an image)
    - CIoU Loss (for gt vs pred bbox comparisons, encourages non-overlapping bboxes to pull closer)
- Bag of specials: marginally increase inference time, but substantially improve performance
    - mish activations: $\text{mish}(x) = x\tanh(\text{softplus}(x))$
    - DIoU non-max suppression (more efficient at picking out best bbox)
    - cross mini-batch normalization (less gpu dependent)
    - dropblock regularization (randomly hide parts of image going into first layer)
- Faster than EfficientDet D0-D2 relative to size
- SOTA on COCO wrt FPS-AP tradeoff as of publication date (April 2020)

### DETR: Detection Transformers (2020)

- One of the first successful transformer-based approaches to object detection
- Removes the need for non-max suppression and anchor boxes since set predictions avoid the worry of overlapping boxes
- Takes in CNN extracted image features and passes them through a transformer encoder-decoder to produce a set of bounding box predictions and classes
- Matching of set predictions with ground truth predictions is done via a bipartite matching loss (essentially BCE + bbox L1/IOU loss against all nonempty output/prediction pairs)
    - Efficient loss calculation on all permutation (y, yhat) pairs is achieved using the Hungarian algorithm
- Bounding box prediction coordinates are made directly, not on the difference relative to some guess
- Interpretability: Self attention layers can be used to reconstruct the feature maps for each transformer layer
- Can be extended to panoptic segmentation by adding a fully convolutional CNN after the decoder output to produce the segmentation masks

<img src=https://production-media.paperswithcode.com/methods/Screen_Shot_2020-07-20_at_9.17.39_PM_ZHS2kmV.png width=600>

### DINO: DETR with Improved DeNoising Anchor Boxes for End-to-End Object Detection (2022)

- Extends DETR by adding in denoising, learned anchor boxes, and a few other tweaks.
- Not a lot out there on this right now. It also gets confused with a SSL model also named DINO.
- Currently at or near the SOTA for COCO object detection.

<img src=https://github.com/IDEACVR/DINO/raw/main/figs/framework.png width=700>

### SimCLR: A Simple Framework for Contrastive Learning of Visual Representations (2020)

- SimCLR is one of the first successful self-supervised approaches to learning representations of images. Its defining feature is the use of contrastive learning to learn visual representations.
- Framework for contrastive learning of visual representations. Learns representations by maximizing agreement between differently augmented views of the same image via a contrastive loss in the latent space.
- Given a particular image $x$, Sim CLR uses (random) data augmentation (random crop, resize, random color distortion, and random Gaussian blur) to produce two augmented versions $\tilde x_1, \tilde x_2$ of the same image. The particular augmentations chosen are essential here.
- An encoder network $f$ then extracts feature vectors $h_i, h_j$ from the two augmented samples. Any usual feature extractor can be used for the encoder (the default was a resnet up to the last pooling layer).
- A projection head $g$ then maps the extracted features to outputs $z_i, z_j$ (default is a linear layer with ReLU). The use of a projection head makes a big difference to performance.
- A contrastive loss is then used for each image $\tilde x_1$ to identify which other example in some set of images is the matching image $\tilde x_2$. The NT-Xent (normalized temperature-scaled cross entropy) loss is used be default, defined by
$$\mathbb{\ell}_{i,j} = -\log\frac{\exp\left(\cos\left(\mathbf{z}_{i}, \mathbf{z}_{j}\right)/\tau\right)}{\sum_{k\neq i}\exp\left(\cos\left(\mathbf{z}_{i}, \mathbf{z}_{k}\right)/\tau\right)},$$
where $\tau$ is some temperature hyperparameter. The loss is computed against all positive pairs in the batch.
- In practice, the set of negative contrastive examples are just the other image augmentations in that batch. This will work better for larger batch sizes.

<img src=https://production-media.paperswithcode.com/methods/Screen_Shot_2020-07-02_at_4.31.34_PM_7zlWDQE.png width=350>

### CoCa: Contrastive Captioners are Image-Text Foundation Models (2022)

- Uses an encoder-decoder transformer to train a semi-supervised (multimodal) image-to-text model for both images and text
- Images feed into the encoder, which uses cross attention to interact with the decoder, which takes in and outputs text
- A combination of contrastive loss and captioning loss is used
- Currently at or near the SOTA on several TTI/ITT tasks, as well as Imagenet Top 1

<img src=https://pbs.twimg.com/media/FR9HjVcXEAAxX9f.jpg:large width=600>

### Diffusion Models: Multiple Papers (2020s)

- Diffusion Models (also called Denoising Diffusion Probabilistic Models, or DDPMs) are generative models that seek to generate images from pure Gaussian noise.
- They work by destroying training data through the successive addition of Gaussian noise, and then learning to recover the data by reversing this noising process. 
- At inference, we can use diffusion model to generate data by simply passing randomly sampled noise through this learned denoising process.
- DMs use a Markov Chain approach to noising/denoising an image. Given an image $x=x_0$, a sequence of $T$ noising steps is performed in succession to produce pure noise via some distribution $q$: $x_0 \rightarrow x_1 \rightarrow \cdots \rightarrow x_T$.
- The model then tries to learn the *reverse* direction of the Markov Chain, taking the pure noise and trying to reconstruct the original image, i.e. predicting the reversed distribution $p_{\theta}$ for the sequence
$x_T \rightarrow x_{T-1} \rightarrow \cdots \rightarrow x_0$.
- Typically, we choose each $q(x_t|x_{t-1}) \equiv \mathcal{N}\big(x_t|\sqrt{1-\beta_t}x_{t-1},\beta_t I\big)$ for each $t=0,\cdots,T$. The variances $\beta_t$ are defined according to some schedule, with $\beta_0=0$ and steadily increasing to $\beta_T=1$. Each step $x_t$ is assumed to depend only on the previous step $x_{t-1}$, hence the joint probability is given by $q(x_0,\cdots,x_T) = q(x_0)\prod_{t=1}^T q(x_t|x_{t-1}).$
- If we can thus approximate the distribution $q$ in the *backward* direction, then we can take in pure $x_T \sim \mathcal{N}(0,I)$ Gaussian noise and go through the sequence of steps to generate the image distribution for $x_0$. The reverse distribution $q(x_{t-1}|x_t)$ however depends on the entire data distribution (unlike the forward distribution), hence we try to model it with a neural network to learn the approximating backward distribution $p_{\theta}(x_{t-1}|x_t)$.
- We assume that $p_{\theta}(x_{t-1}|x_t) \equiv \mathcal{N}\big(x_{t-1}|\mu_{\theta}(x_t,t),\Sigma_{\theta}(x_t,t)\big)$, and use a neural network to try to learn the means $\mu_{\theta}(x_t,t)$ and covariances $\Sigma_{\theta}(x_t,t)$. This Gaussian assumption is a strong assumption, and is only reasonable for a large number of time steps $T$.
- It turns out to be much more practical to model the means by splitting off the noise term $\varepsilon_{\theta}(x_t,t)$ at each step via
$$\mu_{\theta}(x_t,t) = \frac{1}{\sqrt{1-\beta_t}}\bigg(x_{t} - \frac{\beta_t}{\sqrt{1-\overline{\alpha}_t}} \varepsilon_{\theta}(x_t,t)\bigg),$$
$$\overline{\alpha}_t = (1-\beta_1)\cdots (1-\beta_t),$$
and trying to learn the noises $\varepsilon_{\theta}(x_t,t)$ instead.
- Similarly, it's practical to model the covariances either by assuming they're fixed via some $\Sigma_{\theta}(x_t,t)=\sigma_t I$, or to model the covariances by learning interpolation vectors $v$ such that 
$$\Sigma_{\theta}(x_t,t)=\exp\big(v\log \beta_t + (1-v)\log\tilde\beta_t\big),$$
where 
$$\tilde{\beta}_t = \frac{1 - \overline{\alpha}_{t-1}}{1 - \overline{\alpha}_t} \beta_t.$$
- The loss to be minimized during training is the combination of the MSE between the true (sampled) noise $\varepsilon$ and the predicted noise $\varepsilon_{\theta}(x_t,t)$, and the variational lower bound $L_{vlb}$:
$$L = \big\langle ||\varepsilon_{\theta}(x_t,t) - \varepsilon||^2 \big\rangle + \lambda L_{vlb}.$$
Note the average is taken across all time steps $t$ and all inputs $x_0$. The VLB loss is given by the sum of KL divergences between the true and predicted distributions at each time step.
- The forward noising steps can be more efficiently generated by noting that
$$q(x_t|x_0) = \mathcal{N}\big(x_t|\;\overline{\alpha}_t^{1/2}x_0,\,(1-\overline{\alpha}_t)I \big).$$
This is more efficient because during training we won't want to generate *all* noises $t=1,\cdots,T$, but only randomly sample a $t$ and noise to that step.
- The goal of the neural network, then, is to take a noised image $x_t$ and try to predict how much noise $\varepsilon_{\theta}(x_t,t)$ was applied to it to get from the original image $x_0$ to $x_t$. Typically for images, U-Nets are used to do this, i.e. to map the image $x_t$ to the noised image $\varepsilon_{\theta}(x_t,t)$.
- Unlike GANs, DMs are typically reasonably easy to train. They usually converge, and don't require adversarial training.

<img src=https://www.assemblyai.com/blog/content/images/2022/05/image.png width=600>
<img src=https://www.assemblyai.com/blog/content/images/2022/05/image-1.png width=600>
<img src=https://huggingface.co/blog/assets/78_annotated-diffusion/training.png width=400>